In [ ]:
!nvidia-smi

Thu Mar  6 04:05:03 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   49C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!git clone https://github.com/microsoft/MoGe.git

Cloning into 'MoGe'...
remote: Enumerating objects: 218, done.
remote: Counting objects: 100% (46/46), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 218 (delta 34), reused 21 (delta 21), pack-reused 172 (from 1)
Receiving objects: 100% (218/218), 4.13 MiB | 8.82 MiB/s, done.
Resolving deltas: 100% (80/80), done.


In [ ]:
%cd MoGe

/content/MoGe/MoGe


In [ ]:
!pip install -r requirements.txt

In [ ]:
import cv2
import torch
import trimesh
import numpy as np
from PIL import Image
from moge.model import MoGeModel
import utils3d

# Set up device
device = torch.device("cuda")

# Load the model from huggingface hub
model = MoGeModel.from_pretrained("Ruicheng/moge-vitl").to(device).eval()

# Read the input image and convert to tensor
input_image = cv2.cvtColor(cv2.imread("/content/lioness.png"), cv2.COLOR_BGR2RGB)
height, width = input_image.shape[:2]

# Convert image to tensor
input_image_tensor = torch.tensor(input_image / 255, dtype=torch.float32, device=device).permute(2, 0, 1)

# Infer
output = model.infer(input_image_tensor, resolution_level=9, apply_mask=True)

# Extract outputs and convert to NumPy
points = output['points'].cpu().numpy()
depth = output['depth'].cpu().numpy()
mask = output['mask'].cpu().numpy()

# Compute normals
normals, normals_mask = utils3d.numpy.points_to_normals(points, mask=mask)

# Generate mesh
faces, vertices, vertex_colors, vertex_uvs = utils3d.numpy.image_mesh(
    points,
    input_image.astype(np.float32) / 255,  # Using the original NumPy array
    utils3d.numpy.image_uv(width=width, height=height),
    mask=mask & ~(
        utils3d.numpy.depth_edge(depth, rtol=0.03, mask=mask) &
        utils3d.numpy.normals_edge(normals, tol=5, mask=normals_mask)
    ),
    tri=True
)

# Adjust vertices and UVs
vertices, vertex_uvs = vertices * [1, -1, -1], vertex_uvs * [1, -1] + [0, 1]

# Create trimesh with texture
mesh = trimesh.Trimesh(
    vertices=vertices * [-1, 1, -1],  # Adjust for 3D viewer
    faces=faces,
    visual=trimesh.visual.texture.TextureVisuals(
        uv=vertex_uvs,
        material=trimesh.visual.material.PBRMaterial(
            baseColorTexture=Image.fromarray(input_image),
            metallicFactor=0.5,
            roughnessFactor=1.0
        )
    ),
    process=False
)

# # Export the mesh
# # mesh.export('output_model.glb')
mesh.export('output_model.ply')

print("3D model exported as output_model.ply")

3D model exported as output_model.ply
